<a href="https://colab.research.google.com/github/wingmoe21/tech_innovators/blob/main/backend_basmah.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install git+https://github.com/m-bain/whisperx.git

  Cloning https://github.com/m-bain/whisperx.git to /tmp/pip-req-build-r1l5b3pb
  Running command git clone --filter=blob:none --quiet https://github.com/m-bain/whisperx.git /tmp/pip-req-build-r1l5b3pb
  Resolved https://github.com/m-bain/whisperx.git to commit d97cdb7bcf302fb3e1651321a5935f90594e994c
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 105.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.6/208.6 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 92.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━

In [2]:
%pip install moviepy ffmpeg

  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6082 sha256=4ad474a0b55ad69ea3df64151fb9ebf5f1def92bedf9b29ce090d6c5a67c9ecb
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg


In [3]:
!pip install opencv-python

In [4]:
!pip install pytesseract io pdf2image opencv-python

ERROR: Could not find a version that satisfies the requirement io (from versions: none)
ERROR: No matching distribution found for io


In [5]:
!pip install scikit-image

In [6]:
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 56.0 MB/s eta 0:00:00


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
from PIL import Image
import numpy as np
from skimage.metrics import structural_similarity as ssim
import io
import multiprocessing
import matplotlib.pyplot as plt
import fitz
import whisperx
from moviepy.editor import VideoFileClip

def convert_mp4_to_mp3(input_file, output_file):
    video_clip = VideoFileClip(input_file)
    audio_clip = video_clip.audio
    audio_clip.write_audiofile(output_file)
    audio_clip.close()

def transcribe_and_align(input_audio_path, output_file_path, batch_size=4, device='cuda', compute_type='cuda'):
    # Transcribe with original whisper (batched)
    model = whisperx.load_model("large-v2", device, compute_type= "float16")
    audio = whisperx.load_audio(input_audio_path)
    transcribe_result = model.transcribe(audio, batch_size=batch_size)

    # Align whisper output
    align_model, metadata = whisperx.load_align_model(language_code=transcribe_result["language"], device=device)
    align_result = whisperx.align(transcribe_result["segments"], align_model, metadata, audio, device, return_char_alignments=False)

    # Save the aligned segments to a text file
    with open(output_file_path, 'w') as f:
        for segment in align_result["segments"]:
            f.write(f"{segment['start']} - {segment['end']}: {segment['text']}\n")

    print(f"Aligned segments saved to {output_file_path}")

def are_frames_similar(frame_pair):
    frame1, frame2 = frame_pair
    ssi_index, _ = ssim(frame1, frame2, full=True)
    return ssi_index > similarity_threshold

def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frames.append(gray_frame)

    cap.release()

    with multiprocessing.Pool() as pool:
        frame_pairs = zip(frames[:-1], frames[1:])
        similar_frames = pool.map(are_frames_similar, frame_pairs)

    unique_slides = []

    for i, is_similar in enumerate(similar_frames):
        if not is_similar:
            slide_image = Image.fromarray(frames[i + 1])
            unique_slides.append(slide_image)

    return unique_slides

def save_slides_to_pdf(unique_slides, pdf_path):
    pdf_doc = fitz.open()

    for slide in unique_slides:
        img_byte_arr = io.BytesIO()
        slide.save(img_byte_arr, format='PNG')
        img_byte_arr = img_byte_arr.getvalue()
        img = fitz.open("png", img_byte_arr)
        rect = img[0].rect
        pdfbytes = img.convert_to_pdf()
        img.close()
        imgPDF = fitz.open("pdf", pdfbytes)
        page = pdf_doc.new_page(width=rect.width, height=rect.height)
        page.show_pdf_page(rect, imgPDF, 0)

    pdf_doc.save(pdf_path)
    pdf_doc.close()

    print(f"PDF with extracted slides created at: {pdf_path}")

if __name__ == "__main__":
    # Paths
    video_path = '/content/drive/MyDrive/Lectures/Next Greater Element _ GeeksforGeeks.mp4'
    audio_file = 'output.mp3'
    output_text_file = 'aligned_segments.txt'
    output_pdf_path = 'extracted_slides.pdf'

    # Tolerance to decide whether two images are similar
    similarity_threshold = 0.95

    # Convert video to audio
    convert_mp4_to_mp3(video_path, audio_file)

    # Transcribe and align audio
    transcribe_and_align(audio_file, output_text_file)

    # Process video frames and save slides to PDF
    unique_slides = process_video(video_path)
    save_slides_to_pdf(unique_slides, output_pdf_path)